In [ ]:
import torch
import torchvision
import cv2
import numpy as np
import os
from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from random import choice, choices, shuffle
import re
from ipywidgets import IntProgress
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import albumentations as A
from albumentations.pytorch.transforms import ToTensor, ToTensorV2


BOX_COLOR = (0, 0, 255)
TEXT_COLOR = (255, 255, 255)
TRAIN_IMG_DIR = "../input/global-wheat-detection/train"

In [ ]:
#show 1 ảnh
def plot_img(img, size=(7,7), is_rgb=False):
    plt.figure(figsize=size)
    if is_rgb:
        plt.imshow(img)
    else:
        plt.imshow(img[:,:,::-1])
    plt.show()
    
    
#show nhiều ảnh
def plot_imgs(imgs, cols=5, size=7, is_rgb=False):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        fig.add_subplot(rows, cols, i+1)
        if is_rgb:
            plt.imshow(img)
        else:
            plt.imshow(img[:,:,::-1])
    plt.show()
    
    
# vẽ bounding box lên ảnh
def visualize_bbox(img, boxes, thickness=3, color=BOX_COLOR):
    img_copy = img.copy()
    for box in boxes:
        img_copy = cv2.rectangle(
            img_copy,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, thickness)
    return img_copy


# vẽ bounding box lên ảnh
def load_img(img_id, folder=TRAIN_IMG_DIR):
    img_fn = f"{folder}/{img_id}.jpg"
    img = cv2.imread(img_fn).astype(np.float32)
    img /= 255.0
    return img

In [ ]:
#chuyển đổi cặp [imgs, targets] sang dạng tensor theo device cpu/gpu
def data_to_device(images, targets, device=torch.device("cuda")):
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    return images, targets


def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r


# đọc data từ file csv
# output là 1 list chứa thông tin về các ảnh
# mỗi phần tử bao gồm 1 image_id và 1 list các bounding box
def read_data_in_csv(csv_path="./wheat-dataset/train.csv"):
    df = pd.read_csv(csv_path)
    df['x'], df['y'],  df['w'], df['h'] = -1, -1, -1, -1
    df[['x', 'y', 'w', 'h']] = np.stack(df['bbox'].apply(lambda x: expand_bbox(x)))
    df.drop(columns=['bbox'], inplace=True)
    df['x'] = df['x'].astype(np.float)
    df['y'] = df['y'].astype(np.float)
    df['w'] = df['w'].astype(np.float)
    df['h'] = df['h'].astype(np.float)
    objs = []
    img_ids = set(df["image_id"])
    
    for img_id in tqdm(img_ids):
        records = df[df["image_id"] == img_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        area = boxes[:,2]*boxes[:,3]
        boxes[:,2] = boxes[:,0] + boxes[:,2]
        boxes[:,3] = boxes[:,1] + boxes[:,3]

        obj = {
            "img_id": img_id,
            "boxes": boxes,
            "area":area
        }
        objs.append(obj)
    return objs


class WheatDataset(Dataset):
    def __init__(self, data, img_dir ,transform=None):
        self.data = data
        self.img_dir = img_dir
        self.transform = transform
        
    def __getitem__(self, idx):
        img_data = self.data[idx]
        bboxes = img_data["boxes"]
        box_nb = len(bboxes)
        labels = torch.ones((box_nb,), dtype=torch.int64)
        iscrowd = torch.zeros((box_nb,), dtype=torch.int64)
        img = load_img(img_data["img_id"], self.img_dir)
        area = img_data["area"]
        if self.transform is not None:
            sample = {
                "image":img,
                "bboxes": bboxes,
                "labels": labels,
                "area": area
            }
            sample = self.transform(**sample)
            img = sample['image']
            area = sample["area"]
            bboxes = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        target = {}
        target['boxes'] = bboxes.type(torch.float32)
        target['labels'] = labels
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = iscrowd
        target["image_id"] = torch.tensor([idx])
        return img, target
        
    def __len__(self):
        return len(self.data)
    

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# #load data form csv file
# data = read_data_in_csv('../input/global-wheat-detection/train.csv')
# shuffle(data)
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# # tạo transform cho dataset - các biến đổi để augmentation data
# train_transform = A.Compose(
#     [A.Flip(0.5), ToTensorV2(p=1.0)],
#     bbox_params={
#         "format":"pascal_voc",
#         'label_fields': ['labels']
# })

# # khởi tạo Dataset và Dataloader
# train_dataset = WheatDataset(data, img_dir=TRAIN_IMG_DIR, transform=train_transform)
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=8,
#     shuffle=True,
#     num_workers=2,
#     collate_fn=collate_fn)


In [ ]:
# from torch.optim.lr_scheduler import _LRScheduler
# from torch.optim.lr_scheduler import ReduceLROnPlateau


# class GradualWarmupScheduler(_LRScheduler):
#     def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
#         self.multiplier = multiplier
#         if self.multiplier < 1.:
#             raise ValueError('multiplier should be greater thant or equal to 1.')
#         self.total_epoch = total_epoch
#         self.after_scheduler = after_scheduler
#         self.finished = False
#         super(GradualWarmupScheduler, self).__init__(optimizer)

#     def get_lr(self):
#         if self.last_epoch > self.total_epoch:
#             if self.after_scheduler:
#                 if not self.finished:
#                     self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
#                     self.finished = True
#                 return self.after_scheduler.get_last_lr()
#             return [base_lr * self.multiplier for base_lr in self.base_lrs]

#         if self.multiplier == 1.0:
#             return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
#         else:
#             return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

#     def step_ReduceLROnPlateau(self, metrics, epoch=None):
#         if epoch is None:
#             epoch = self.last_epoch + 1
#         self.last_epoch = epoch if epoch != 0 else 1  # ReduceLROnPlateau is called at the end of epoch, whereas others are called at beginning
#         if self.last_epoch <= self.total_epoch:
#             warmup_lr = [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
#             for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
#                 param_group['lr'] = lr
#         else:
#             if epoch is None:
#                 self.after_scheduler.step(metrics, None)
#             else:
#                 self.after_scheduler.step(metrics, epoch - self.total_epoch)

#     def step(self, epoch=None, metrics=None):
#         if type(self.after_scheduler) != ReduceLROnPlateau:
#             if self.finished and self.after_scheduler:
#                 if epoch is None:
#                     self.after_scheduler.step(None)
#                 else:
#                     self.after_scheduler.step(epoch - self.total_epoch)
#                 self._last_lr = self.after_scheduler.get_last_lr()
#             else:
#                 return super(GradualWarmupScheduler, self).step(epoch)
#         else:
#             self.step_ReduceLROnPlateau(metrics, epoch)

In [ ]:


# num_classes = 2
# num_epochs = 5
# iters = 1
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=False)
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

# scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs-1)
# scheduler = GradualWarmupScheduler(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

# model.to(device)

In [ ]:
# # tiến hành train model
# for epoch in range(num_epochs):
#     scheduler.step(epoch)
#     model.train()
#     for images, targets in train_loader:
#         images, targets = data_to_device(images, targets)
#         loss_dict = model(images, targets)
#         losses = sum(loss for loss in loss_dict.values())
#         loss_value = losses.item()
        
#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()
        
#         iters += 1
#         # show loss per 30 iteration
#         if iters%30 == 0:
#             print(f"Iteration #{iters} loss: {loss_value}")
            
# #         # để đơn giản, ta save model mỗi 90 iteration
# #         if iters%90 == 0:
# #             model_path = f"./saved_model/model_{iters}_{round(loss_value, 2)}.pth"
# #             torch.save(model.state_dict(), model_path)
# #             model.train()

In [ ]:
# torch.save(model.state_dict(), './model_lr_5epoch.h5')

=========================================================================

In [ ]:
class WheatTestDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
def get_test_transform():
    return A.Compose([
        # A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])

In [ ]:
# load a model; pre-trained on COCO
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
# in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
# model.load_state_dict(torch.load('../input/fatercnnresnet152102410epochfold1/fastercnnresnet152_10ep_fold1_ap60.h5'))
model = torch.load('../input/new-gwd2021-resnet50/model_ep-7_ap-0.6110084455701318')

model.eval()

x = model.to(device)

In [ ]:
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
# from torchvision.models.detection.faster_rcnn import FasterRCNN
# from torchvision.models.detection.backbone_utils import BackboneWithFPN

# def fasterrcnn_resnet_fpn(backbone_name='resnet152', progress=True, num_classes=91, pretrained=True, pretrained_backbone=True, **kwargs):
#     if backbone_name == 'resnet50':
#         model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=pretrained)
#     elif backbone_name in ['resnet101', 'resnet152']:
#         backbone = resnet_fpn_backbone(backbone_name, pretrained_backbone)
#     else:
#         backbone = my_resnet_fpn_backbone(backbone_name, pretrained_backbone)
#     model = FasterRCNN(backbone, num_classes, **kwargs)
#     return model

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # model = fasterrcnn_resnet_fpn(backbone_name='resnet152', pretrained=True, pretrained_backbone=True)
# # in_features = model.roi_heads.box_predictor.cls_score.in_features
# # model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
# # model.load_state_dict(torch.load('../input/testmodelfastercnndungnb/model-frcnn-dungnb.h5'))

# model1 = torch.load('../input/fasterrcnn20epfull1024folds1/model_ep-19_trainloss-1.0128233870248047')
# model1.eval()

# model2 = torch.load('../input/testdungnbnew/model-frcnn-dungnb-new.h5')
# model2.eval()

# model1 = model1.to(device)
# model2 = model2.to(device)

In [ ]:
# ££££££££££££££££££££££££££££££££££££££3

In [ ]:
# detection_threshold = 0.5
# results = []

# for images, image_ids in test_data_loader:

#     images = list(image.to(device) for image in images)
#     output1 = model1(images)
#     output2 = model2(images)
#     break
#     for i, image in enumerate(images):

#         boxes = outputs[i]['boxes'].data.cpu().numpy()
#         scores = outputs[i]['scores'].data.cpu().numpy()
        
#         boxes = boxes[scores >= detection_threshold].astype(np.int32)
#         scores = scores[scores >= detection_threshold]
#         image_id = image_ids[i]
        
#         boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
#         boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
#         result = {
#             'image_id': image_id,
#             'PredictionString': format_prediction_string(boxes, scores)
#         }

        
#         results.append(result)

In [ ]:
# ! pip install ensemble-boxes==1.0.4

In [ ]:
# from ensemble_boxes import weighted_boxes_fusion


In [ ]:
# boxe1 = output1[0]['boxes'].cpu().detach().numpy()
# boxe2 = output2[0]['boxes'].cpu().detach().numpy()

In [ ]:
# temp1 = pd.DataFrame(boxe1)
# temp2 = pd.DataFrame(boxe2)

In [ ]:
# temp[0][3]

In [ ]:
# boxes = []
# for i in range(len(temp1)):
#     boxes.append([temp1[i][0], temp1[i][1], temp1[i][2], temp1[i][3]])
# len(boxes)

In [ ]:
# boxes, scores, labels

In [ ]:
# boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=0.5, skip_box_thr=0.32)
# boxes = np.array(boxes)
# scores = np.array(scores)

In [ ]:
# ££££££££££££££££££££££££££££££££££££££££££££££

In [ ]:
test_df = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')
test_df.shape

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = WheatTestDataset(test_df, '../input/global-wheat-detection/test', get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
detection_threshold = 0.5
results = []

for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }

        
        results.append(result)

In [ ]:
# results[0:2]


In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

In [ ]:
test_df.to_csv('./submission.csv',index=False)

In [ ]:
# sample = images[2].permute(1,2,0).cpu().numpy()
# boxes = outputs[2]['boxes'].data.cpu().numpy()
# scores = outputs[2]['scores'].data.cpu().numpy()

# boxes = boxes[scores >= detection_threshold].astype(np.int32)


# fig, ax = plt.subplots(1, 1, figsize=(16, 8))

# for box in boxes:
#     cv2.rectangle(sample,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 2)
    
# ax.set_axis_off()
# ax.imshow(sample)

In [ ]:
# torch.save(model,'./model.h5')

In [ ]:
# model1 = torch.load('../input/testmodelfastercnndungnb/model-frcnn-dungnb.h5')

In [ ]:
# model1

In [ ]:
# model1 = torch.load('../input/testmodel/model.h5')